In [73]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
import numpy as np

In [74]:
### import data 
df = pd.read_csv('GISP20002019.csv')
print(df.head())

df.iloc[:, 5:len(df.columns)] = df.iloc[:, 5:len(df.columns)].apply(pd.to_numeric, errors='coerce')
## errors = coerce converts missing data to NaN
print(df.head())


   Obs CLINIC  YEAR         GENDERSP  TOTAL Susceptible TetI PenI PenI_TetI  \
0    1    ALB  2000              MSW    143          10    2   36        68   
1    2    ALB  2000              MSM      9           .    .    1         5   
2    3    ALB  2000             MSMW      5           .    .    2         3   
3    4    ALB  2000  Oth/Unk/Missing      4           .    1    .         1   
4    5    ALB  2001              MSW    127          12    3   40        57   

  TetR  ... AziRS_TetR_CipI_PenI CipR CipR_TetR AziRS_CipI_PenI_TetI  \
0    2  ...                    .    .         .                    .   
1    .  ...                    .    .         .                    .   
2    .  ...                    .    .         .                    .   
3    .  ...                    .    .         .                    .   
4    1  ...                    .    .         .                    .   

  CfxRS_CipR_TetR_PenI AziRS_TetR AziRS_CipR_PenR_TetI  \
0                    .          . 

/var/folders/3f/vvt02dh16jddhc3q1tl6f3zh0000gn/T/ipykernel_92921/1715626302.py:5: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:, 5:len(df.columns)] = df.iloc[:, 5:len(df.columns)].apply(pd.to_numeric, errors='coerce')


In [75]:
## narrow down from Revant's code, only look at 
drugs = ["CIP"]
drugs_name = {
    'CIP': 'Ciprofloxacin',}

drugsDict = {}
for drug in drugs:
    drugsDict[drug] = list(df.columns)

#print(drugsDict)
print(drugsDict["CIP"])


['Obs', 'CLINIC', 'YEAR', 'GENDERSP', 'TOTAL', 'Susceptible', 'TetI', 'PenI', 'PenI_TetI', 'TetR', 'TetR_PenI', 'PenR', 'PenR_TetI', 'PenR_TetR', 'CipI_PenI_TetI', 'TetR_CipI', 'AziRS_PenI_TetI', 'CipI_TetI', 'CipR_PenR_TetR', 'CipR_PenI_TetI', 'CipR_TetR_PenI', 'CipR_PenI', 'CipR_PenR_TetI', 'AziRS_CipR_PenR_TetR', 'PenR_CipI_TetI', 'PenR_TetR_CipI', 'CipR_PenR', 'TetR_CipI_PenI', 'AziRS_CipR_TetR_PenI', 'AziRS_TetR_PenI', 'AziRS_PenR_TetR', 'AziRS_PenR_TetI', 'CipI_PenI', 'CfxRS_CipR_PenR_TetR', 'CfxRS_PenI_TetI', 'AziRS', 'AziRS_TetI', 'AziRS_PenI', 'CfxRS_CipI_PenI_TetI', 'AziRS_CipR_PenI_TetI', 'CfxRS_TetR_PenI', 'CipR_TetI', 'CipI', 'AziRS_PenR', 'PenR_CipI', 'AziRS_TetR_CipI_PenI', 'CipR', 'CipR_TetR', 'AziRS_CipI_PenI_TetI', 'CfxRS_CipR_TetR_PenI', 'AziRS_TetR', 'AziRS_CipR_PenR_TetI', 'CroRS_CfxRS_CipR_PenR_TetR', 'CroRS_CfxRS_PenR_TetI', 'CfxRS_PenR_TetR']


In [76]:
#print(drugsDict["CIP"][18])
#print("CipR" not in drugsDict["CIP"][18])
#print("CLINIC" not in drugsDict["CIP"][1])

for variable in range(len(list(df.columns))):
    if "CipR" not in drugsDict["CIP"][variable]: ## so this is saying that if CipR is not in the variable (comparing susbsections of strings?)
        drugsDict["CIP"][variable] = None
   
for drug in drugs:
    drugsDict[drug] = list(filter(None, drugsDict[drug]))
    df["Total"+drug] = df[drugsDict[drug]].sum(axis=1, skipna=True)

#print(df.head())
#print(df["TotalCIP"])


0        0.0
1        0.0
2        0.0
3        0.0
4        0.0
        ... 
1965     0.0
1966    14.0
1967    13.0
1968     4.0
1969     1.0
Name: TotalCIP, Length: 1970, dtype: float64


#now by location 
west = ['POR', 'PHX', 'HON', 'SDG', 'SFO', 'ANC', 'SEA', 'DEN', 'LVG', 'ORA', 'LBC', 'SLC', 'LAX']
southwest = ['OKC','MIN', 'ALB', 'DAL']
midwest = ['KCY','CHI', 'PON', 'CIN', 'JAC', 'IND', 'STL','DTR', 'MIL', 'COL', 'CLE']
southeast = ['GRB', 'NOR','WDC','MIA', 'BHM','FBG','ATL', 'RIC']
northeast = ['BUF','BOS', 'CAM', 'NYC', 'BAL', 'PHI']

df['Region'] = df['CLINIC'].apply(lambda x: 
    'West' if (x in west) else (
        'Southwest' if (x in southwest) else(
            'Midwest' if (x in midwest) else(
                'Southeast' if (x in southeast) else(
                    'Northeast' if (x in northeast) else 'Other'))))) 
    

In [ ]:
overall = pd.DataFrame([],[])
for year in list(set(df["YEAR"])):
     df2 = df[df["YEAR"]==year]
     new = {'Year': year}
    
     new['OverallTotal'] = df2["TOTAL"].sum()
    
    
     for drug in drugs:
           new[drug+'_OverallTotal'] = df2["Total"+drug].sum()
           new[drug+'_OverallPct'] = (new[drug+'_OverallTotal']/new['OverallTotal'] )*100
           
           for gender in list(set(df["GENDERSP"])):       
                df3 = df2[df2["GENDERSP"]==gender]
            
                new[gender+"_Total"] = df3["TOTAL"].sum()
                new[gender+"_"+drug+"Total"] = df3["Total"+drug].sum()
                new[gender+"_"+drug+"Pct"] = (new[gender+"_"+drug+"Total"] / new[gender+"_Total"])*100
        
           for region in list(set(df["Region"])):       
                df4 = df2[df2["Region"]==region]
            
                new[region+"_Total"] = df4["TOTAL"].sum()
                new[region+"_"+drug+"_Total"] = df4["Total"+drug].sum()
                new[region+"_"+drug+"_Pct"] = (new[region+"_"+drug+"_Total"]/new[region+"_Total"])*100
    
     overall = overall.append(new, ignore_index = True)
overall["Year"] = overall["Year"].astype(int)